In [1]:
import pandas as pd
import dateutil
import numpy as np
import datetime
import re
pd.options.display.max_columns = 40

In [2]:
all_assignments=pd.read_csv("14532-P540798-InvisibleInstitute_DataInfoforallswornCPDofficers-UPDATEv2.csv",
                            parse_dates=["APPOINTED DATE", "END DATE", "EFFECTIVE DATE"],
                           dtype={"YEAR OF BIRTH": "Int64", "STAR NUMBER": "Int64"})

In [3]:
all_assignments.index.name="assignment_index"

In [4]:
all_assignments = all_assignments.reset_index().set_index("assignment_index", drop=False)

In [5]:
def date_minus_100_years(date):
    return datetime.datetime(date.year-100, date.month, date.day)

def fix_y2k(field):
    all_assignments[field] = all_assignments[field].apply(
        lambda x: date_minus_100_years(x) if not pd.isna(x) and x> pd.datetime.now() else x
    )

In [6]:
fix_y2k("APPOINTED DATE")

fix_y2k("END DATE")

fix_y2k("EFFECTIVE DATE")

In [7]:
all_assignments["MIDDLE INITIAL"].fillna("", inplace=True)

In [8]:
def last_name_key(name):
    if name:
        n = re.sub(r"[^A-Z ]","",name.upper())
        pieces = n.split()
        if pieces[-1] in ["JR", "SR", "I", "II", "III", "IV"]:
            pieces = pieces[:-1]
        return "".join(pieces)
    return ""

In [9]:
all_assignments["last_name_key"] = all_assignments["LAST NAME"].apply(last_name_key)

In [10]:
all_assignments["END DATE"].fillna(pd.Timestamp.max, inplace=True)

In [11]:
all_assignments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35863 entries, 0 to 35862
Data columns (total 13 columns):
assignment_index    35863 non-null int64
FIRST NAME          35863 non-null object
MIDDLE INITIAL      35863 non-null object
LAST NAME           35863 non-null object
YEAR OF BIRTH       35863 non-null Int64
SEX                 35863 non-null object
RACE                35863 non-null object
APPOINTED DATE      35863 non-null datetime64[ns]
STAR TYPE           35863 non-null object
STAR NUMBER         35863 non-null Int64
EFFECTIVE DATE      35863 non-null datetime64[ns]
END DATE            35863 non-null datetime64[ns]
last_name_key       35863 non-null object
dtypes: Int64(2), datetime64[ns](3), int64(1), object(7)
memory usage: 3.9+ MB


In [12]:
keys = ["FIRST NAME", "last_name_key", "APPOINTED DATE", "YEAR OF BIRTH", "RACE", "SEX", "MIDDLE INITIAL"]

In [13]:
cops = all_assignments.groupby(keys).agg({"END DATE": "max", "STAR NUMBER": list, "assignment_index": list})

In [14]:
cops=cops.reset_index().reset_index()

In [15]:
cops.head()

,index,FIRST NAME,last_name_key,APPOINTED DATE,YEAR OF BIRTH,RACE,SEX,MIDDLE INITIAL,END DATE,STAR NUMBER,assignment_index
0,0,A RICHARD,LABRIOLA,1971-10-25,1943,WHITE,M,,2004-03-15 00:00:00.000000000,[15869],[17488]
1,1,AARON,ACEVEDO,2009-12-16,1982,HISPANIC,M,F,2262-04-11 23:47:16.854775807,"[7792, 20009]","[53, 54]"
2,2,AARON,ALLEY,2016-06-27,1985,BLACK,M,V,2262-04-11 23:47:16.854775807,[9516],[516]
3,3,AARON,ANDERSON,2017-10-16,1981,WHITE,M,,2262-04-11 23:47:16.854775807,[6550],[729]
4,4,AARON,ARMSTEAD,2005-12-05,1975,BLACK,M,P,2262-04-11 23:47:16.854775807,[13768],[1024]


In [16]:
cops["YEAR OF BIRTH"]=cops["YEAR OF BIRTH"].astype("Int64")

In [17]:
cops["index"]=cops["index"].astype("Int64")

In [18]:
cops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22916 entries, 0 to 22915
Data columns (total 11 columns):
index               22916 non-null Int64
FIRST NAME          22916 non-null object
last_name_key       22916 non-null object
APPOINTED DATE      22916 non-null datetime64[ns]
YEAR OF BIRTH       22916 non-null Int64
RACE                22916 non-null object
SEX                 22916 non-null object
MIDDLE INITIAL      22916 non-null object
END DATE            22916 non-null datetime64[ns]
STAR NUMBER         22916 non-null object
assignment_index    22916 non-null object
dtypes: Int64(2), datetime64[ns](2), object(7)
memory usage: 2.0+ MB


In [19]:
cpdp_profiles = pd.read_csv("final-profiles.csv",
                            dtype={4:"str", 15: "str", "birth_year": "Int64", "link_UID": "Int64",
                                  "current_status": "Int64", "current_star": "Int64",
                                  "current_unit": "Int64"},
                            parse_dates=["appointed_date", "resignation_date", "start_date", "org_hire_date"]
                           ).set_index("UID", drop=False)

In [20]:
cpdp_profiles.shape

(33693, 20)

In [21]:
cpdp_reduced = cpdp_profiles.loc[:133667].copy() # tail end does not match cpdp online data

In [22]:
cpdp_reduced["UID"]=cpdp_reduced["UID"].astype("Int64")

In [23]:
cpdp_reduced["last_name_key"] = cpdp_reduced["last_name"].apply(last_name_key)

In [24]:
cpdp_reduced["middle_initial"].fillna("", inplace=True)

In [25]:
cpdp_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33667 entries, 100001 to 133667
Data columns (total 21 columns):
UID                 33667 non-null Int64
first_name          33667 non-null object
last_name           33667 non-null object
middle_initial      33667 non-null object
middle_initial2     8 non-null object
suffix_name         1689 non-null object
birth_year          32562 non-null Int64
race                33477 non-null object
gender              33643 non-null object
appointed_date      33594 non-null datetime64[ns]
resignation_date    20194 non-null datetime64[ns]
current_status      31789 non-null Int64
current_star        15364 non-null Int64
current_unit        31606 non-null Int64
current_rank        31473 non-null object
start_date          18971 non-null datetime64[ns]
org_hire_date       19786 non-null datetime64[ns]
profile_count       33667 non-null int64
cleaned_rank        31473 non-null object
link_UID            33667 non-null Int64
last_name_key       33667

In [26]:
merge1=cops.merge(cpdp_reduced, left_on=["FIRST NAME", "last_name_key", "APPOINTED DATE", "YEAR OF BIRTH"],
                    right_on=["first_name", "last_name_key", "appointed_date", "birth_year"],
                   indicator=True, how="outer")

In [27]:
merge1.groupby("_merge").size()

_merge
left_only      2392
right_only    13140
both          20527
dtype: int64

In [28]:
s=merge1[["index"]].groupby("index").size()
merge1[merge1["index"].isin(s[s>1].index)]

,index,FIRST NAME,last_name_key,APPOINTED DATE,YEAR OF BIRTH,RACE,SEX,MIDDLE INITIAL,END DATE,STAR NUMBER,assignment_index,UID,first_name,last_name,middle_initial,middle_initial2,suffix_name,birth_year,race,gender,appointed_date,resignation_date,current_status,current_star,current_unit,current_rank,start_date,org_hire_date,profile_count,cleaned_rank,link_UID,_merge
8333,8333,JAMES,BANSLEY,2009-12-16,1983,WHITE,M,H,2262-04-11 23:47:16.854775807,[8791],[1535],111205,JAMES,BANSLEY,A,NaN,NaN,1983,WHITE,MALE,2009-12-16,NaT,1,10927,5,POLICE OFFICER,2009-12-16,2009-12-16,10.0,POLICE OFFICER,1262,both
8334,8333,JAMES,BANSLEY,2009-12-16,1983,WHITE,M,H,2262-04-11 23:47:16.854775807,[8791],[1535],111206,JAMES,BANSLEY,H,NaN,NaN,1983,WHITE,MALE,2009-12-16,NaT,1,8791,4,POLICE OFFICER,2009-12-16,2009-12-16,12.0,POLICE OFFICER,1263,both
18197,18196,ROBERT,ANDERSON,1970-01-19,1948,WHITE,M,M,2002-06-12 00:00:00.000000000,[8024],[801],124724,ROBERT,ANDERSON,M,NaN,NaN,1948,WHITE,MALE,1970-01-19,2002-06-15,0,NaN,128,POLICE OFFICER,NaT,NaT,3.0,POLICE OFFICER,589,both
18198,18196,ROBERT,ANDERSON,1970-01-19,1948,WHITE,M,M,2002-06-12 00:00:00.000000000,[8024],[801],124725,ROBERT,ANDERSON,J,NaN,NaN,1948,WHITE,MALE,1970-01-19,2009-04-29,0,NaN,16,POLICE OFFICER,NaT,NaT,3.0,POLICE OFFICER,620,both
18219,18217,ROBERT,BARRETT,1950-06-03,1926,WHITE,M,E,2262-04-11 23:47:16.854775807,[2010],[1718],124757,ROBERT,BARRETT,J,NaN,NaN,1926,WHITE,MALE,1950-06-03,1985-08-17,0,NaN,11,LIEUTENANT OF POLICE,NaT,NaT,4.0,LIEUTENANT,1406,both
18220,18217,ROBERT,BARRETT,1950-06-03,1926,WHITE,M,E,2262-04-11 23:47:16.854775807,[2010],[1718],124758,ROBERT,BARRETT,E,NaN,NaN,1926,WHITE,MALE,1950-06-03,1985-09-10,0,NaN,165,SERGEANT OF POLICE,NaT,NaT,3.0,SERGEANT,1401,both


In [29]:
drop_dup_merge = merge1[merge1["index"].isin(s[s>1].index) & (merge1["MIDDLE INITIAL"]!=merge1["middle_initial"])]

In [30]:
merge1.loc[drop_dup_merge.index, "_merge"] = "right_only"

In [31]:
unmatched_left=cops.loc[merge1[merge1["_merge"]=="left_only"].index].copy()

In [32]:
unmatched_right = cpdp_reduced.loc[merge1[merge1["_merge"]=="right_only"]["UID"]].copy()

In [33]:
unmatched_right["middle_initial"].fillna("",inplace=True)

In [34]:
matched_by_name = unmatched_left.merge(unmatched_right,
                    left_on=["FIRST NAME", "last_name_key", "MIDDLE INITIAL"],
                    right_on=["first_name", "last_name_key", "middle_initial"],
                                           indicator=True)

In [35]:
matched_by_name

,index,FIRST NAME,last_name_key,APPOINTED DATE,YEAR OF BIRTH,RACE,SEX,MIDDLE INITIAL,END DATE,STAR NUMBER,assignment_index,UID,first_name,last_name,middle_initial,middle_initial2,suffix_name,birth_year,race,gender,appointed_date,resignation_date,current_status,current_star,current_unit,current_rank,start_date,org_hire_date,profile_count,cleaned_rank,link_UID,_merge
0,3,AARON,ANDERSON,2017-10-16,1981,WHITE,M,,2262-04-11 23:47:16.854775807,[6550],[729],133386,AARON,ANDERSON,,NaN,NaN,NaN,WHITE,MALE,2017-10-16,NaT,NaN,NaN,NaN,NaN,NaT,NaT,1,NaN,33396,both
1,53,ABIGAIL,ANTUNEZ,2018-02-20,1990,WHITE HISPANIC,F,C,2262-04-11 23:47:16.854775807,[19000],[939],133629,ABIGAIL,ANTUNEZ,C,NaN,NaN,NaN,HISPANIC,FEMALE,2018-02-20,NaT,NaN,NaN,NaN,NaN,NaT,NaT,1,NaN,33639,both
2,58,ABRAHAM,CORTEZ,2016-08-19,1991,WHITE HISPANIC,M,P,2262-04-11 23:47:16.854775807,[3849],[6200],133187,ABRAHAM,CORTEZ,P,NaN,NaN,NaN,HISPANIC,MALE,2016-08-19,NaT,NaN,NaN,NaN,NaN,NaT,NaT,1,NaN,33197,both
3,60,ABRAHAM,ESPARZA,2018-02-20,1986,WHITE HISPANIC,M,,2262-04-11 23:47:16.854775807,[19516],[9051],133634,ABRAHAM,ESPARZA,,NaN,NaN,NaN,HISPANIC,MALE,2018-02-20,NaT,NaN,NaN,NaN,NaN,NaT,NaT,1,NaN,33644,both
4,72,ADALBERTO,ROJAS,2017-07-17,1994,WHITE HISPANIC,M,,2262-04-11 23:47:16.854775807,[16017],[27747],132922,ADALBERTO,ROJAS,,NaN,NaN,NaN,HISPANIC,MALE,2017-07-17,NaT,NaN,16017,NaN,NaN,NaT,2017-07-17,2,NaN,32934,both
5,86,ADAM,HEIDEMANN,2018-01-16,1987,WHITE,M,B,2262-04-11 23:47:16.854775807,[3062],[13524],133559,ADAM,HEIDEMANN,B,NaN,NaN,NaN,WHITE,MALE,2018-01-16,NaT,NaN,NaN,NaN,NaN,NaT,NaT,1,NaN,33569,both
6,97,ADAM,NOTA,2018-01-16,1992,WHITE,M,C,2262-04-11 23:47:16.854775807,[2803],[23143],133531,ADAM,NOTA,C,NaN,NaN,NaN,WHITE,MALE,2018-01-16,NaT,NaN,NaN,NaN,NaN,NaT,NaT,1,NaN,33541,both
7,105,ADAM,RIOS,2017-07-17,1993,WHITE HISPANIC,M,,2262-04-11 23:47:16.854775807,[8686],[27080],132912,ADAM,RIOS,,NaN,NaN,NaN,HISPANIC,MALE,2017-07-17,NaT,NaN,8686,NaN,NaN,NaT,2017-07-17,2,NaN,32924,both
8,120,ADAN,BARRAZA,2017-11-16,1990,WHITE HISPANIC,M,,2262-04-11 23:47:16.854775807,[2747],[1690],133245,ADAN,BARRAZA,,NaN,NaN,NaN,HISPANIC,MALE,2017-11-16,NaT,NaN,NaN,NaN,NaN,NaT,NaT,1,NaN,33255,both
9,132,ADMIR,DEUMIC,2017-09-18,1990,WHITE,M,,2262-04-11 23:47:16.854775807,[7160],[7686],133310,ADMIR,DEUMIC,,NaN,NaN,NaN,WHITE,MALE,2017-09-18,NaT,NaN,NaN,NaN,NaN,NaT,NaT,1,NaN,33320,both


# Additional partial match

In [36]:
partial_confident_match = matched_by_name[(matched_by_name["appointed_date"].isna()&
                                           matched_by_name["birth_year"].isna())
               |(matched_by_name["appointed_date"]==matched_by_name["APPOINTED DATE"])
               |(matched_by_name["birth_year"]==matched_by_name["YEAR OF BIRTH"])]

In [37]:
s=partial_confident_match.groupby(["index"]).size()

partial_confident_match=partial_confident_match.drop(partial_confident_match[
    partial_confident_match["index"].isin(s[s>1].index)&
                       (partial_confident_match["appointed_date"].isna())].index)

In [38]:
len(partial_confident_match)

527

# Generate merge df

In [39]:
merge_table = merge1[merge1["_merge"]=="both"][["index", "UID"]].append(
    partial_confident_match[["index", "UID"]],
    ignore_index=True)

In [40]:
merge_table.shape

(21051, 2)

In [41]:
merge_table["UID"].is_unique

True

In [42]:
merge_table["index"].is_unique

True

In [43]:
len(cops)-len(merge_table)  # number cops not in cpdp

1865

# store tables

In [44]:
col = []
for idx, val in cops["assignment_index"].iteritems():
    for v in val:
        col+=[(idx, v)]
assignment_cop_table = pd.DataFrame.from_records(col, columns=["cops_index","assignment_index"])

In [45]:
assignment_cop_table

,cops_index,assignment_index
0,0,17488
1,1,53
2,1,54
3,2,516
4,3,729
5,4,1024
6,5,3882
7,6,4541
8,6,4542
9,6,4543


In [46]:
assignment_cop_table.to_csv("2_out_historic_roster_grouped_key.csv", index=False)

In [47]:
cops.to_csv("2_out_CPD_historic_roster_grouped.csv", index=False)

In [48]:
merge_table.to_csv("2_out_merge_cops_cpdp.csv", index=False)

In [49]:
all_assignments.index.name=""

In [50]:
all_assignments_grouped_key=all_assignments.merge(assignment_cop_table, on = "assignment_index",
                                                  left_index=False)

In [51]:
all_assignments_grouped_key.to_csv("2_out_CPD_historic_assignments_grouped.csv", index=False)